## ============   Decision Trees    ===============

In [71]:
use_one_hot_encoding = 0    # i do not understand, why one hot encoding can be useful here
                            # it makes more sense with neural network
use_post_pruning     = 0    # = 1 avoids overfitting

assert(use_one_hot_encoding == 0   or    use_one_hot_encoding == 1)
assert(use_post_pruning     == 0   or    use_post_pruning     == 1)  

### Loading Data

In [69]:
import pandas as pd
df = pd.read_csv('dataset_cricket_match/train.csv')
print (id(df))
df = df.drop('Unnamed: 0', axis = 1)
print (id(df))
ndf = df
print (id(df))
df.head(5)

140300225451728
140300542944160
140300542944160


,team,opp,host,year,month,toss,day_match,bat_first,format,fow,score,rpo,result
0,australia,south_africa,sri_lanka,2012,sep,1,0,0,1,5,146,7.30,1
1,india,australia,india,2020,jan,0,0,1,0,6,340,6.80,1
2,canada,scotland,scotland,2009,jul,1,1,0,0,4,286,5.72,1
3,australia,england,australia,1987,jan,1,1,1,0,6,225,4.50,1
4,new_zealand,pakistan,uae,2009,nov,0,0,0,1,5,153,7.65,0


### STATS

In [27]:
print ("number of rows         : ", df.shape[0])
print ("number of features     : ", df.shape[1]-1)  # excluding the class labels
print ("number of class labels : ", 2)
df.info()

number of rows         :  7827
number of features     :  12
number of class labels :  2
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7827 entries, 0 to 7826
Data columns (total 13 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   team       7827 non-null   object 
 1   opp        7827 non-null   object 
 2   host       7827 non-null   object 
 3   year       7827 non-null   int64  
 4   month      7827 non-null   object 
 5   toss       7827 non-null   int64  
 6   day_match  7827 non-null   int64  
 7   bat_first  7827 non-null   int64  
 8   format     7827 non-null   int64  
 9   fow        7827 non-null   int64  
 10  score      7827 non-null   int64  
 11  rpo        7827 non-null   float64
 12  result     7827 non-null   int64  
dtypes: float64(1), int64(8), object(4)
memory usage: 795.1+ KB


### Compute Entropy

In [60]:
print (id(df))
df.iloc[0,5] = 16
df.head(5)
print (id(df))

140300514480032
140300514480032


In [62]:
print (id(df))
df = df.loc[df["team"] == "india"]
print (id(df))
df.head(5)

140300514480032
140300791484128


,team,opp,host,year,month,toss,day_match,bat_first,format,fow,score,rpo,result
1,india,australia,india,2020,jan,0,0,1,0,6,340,6.80,1
6,india,pakistan,india,1999,mar,0,1,0,0,9,278,5.56,0
7,india,australia,australia,1981,jan,1,0,0,0,8,242,4.84,0
9,india,sri_lanka,india,2014,nov,0,0,0,0,8,274,5.48,1
19,india,west_indis,india,1988,jan,1,1,0,0,10,196,4.04,0


In [64]:
def compEntropy(group_df, attribute):
    print (id(group_df))
    group_df = group_df.groupby(attribute).sum()
    print (id(group_df))
    #print (group_df)
    return group_df

group_df = df.copy()
print (id(group_df))
group_df = compEntropy(group_df,"team" )
print (id(group_df))
#df["team"].value_counts().to_dict()

140300543440832
140300543440832
140300542901264
140300542901264


##### JUNK (Entropy Example)

In [12]:
import math

entropy_1     = -1*(1/5*math.log(1/5,2) +  4/5*math.log(4/5,2))         # base 2
entropy_2     = -1*(8/20*math.log(8/20,2) +  12/20*math.log(12/20,2))   # base 2
entropy_full  =  1/3*entropy_1 + 2/3*entropy_2

print ("entropy for [8|2]  :",  entropy_1)
print ("entropy for [12|8] :",  entropy_2)
print ("total entropy      :",  entropy_full)

entropy for [8|2]  : 0.7219280948873623
entropy for [12|8] : 0.9709505944546686
total entropy      : 0.8879430945988998
